In [1]:
import symforce

symforce.set_symbolic_api("sympy")
symforce.set_log_level("warning")

from symforce.notebook_util import display
import symforce.symbolic as sf

SO3d = sf.Rot3.from_angle_axis(sf.pi/2, sf.V3(0, 0, 1))
q = SO3d.q
R = SO3d.to_rotation_matrix()
# To tangent is what the book and sophus calls .log()
so3 = sf.V3(SO3d.to_tangent())
so3.transpose()

⎡      π⎤
⎢0  0  ─⎥
⎣      2⎦

In [2]:
S = so3.skew_symmetric(so3)
S

⎡   -π    ⎤
⎢0  ───  0⎥
⎢    2    ⎥
⎢         ⎥
⎢π        ⎥
⎢─   0   0⎥
⎢2        ⎥
⎢         ⎥
⎣0   0   0⎦

In [3]:
def skew_to_vec(M: sf.M33)->sf.Vector3:
    return sf.Vector3(M[2, 1], M[0, 2], M[1, 0])

skew_to_vec(S).transpose()

⎡      π⎤
⎢0  0  ─⎥
⎣      2⎦

In [4]:
# Update by perturbation model: 
update_so3 = sf.V3(1e-4, 0, 0)  # A small update
SO3_updated =  (sf.Rot3.from_tangent(update_so3) * SO3d).to_rotation_matrix()
display(SO3_updated)

⎡         0           -1.0           0          ⎤
⎢                                               ⎥
⎢    0.999999995       0    -9.99999998333333e-5⎥
⎢                                               ⎥
⎣9.99999998333333e-5   0        0.999999995     ⎦

In [5]:
# For SE(3)
t = sf.V3(1,0,0)
SE3 = sf.Pose3_SE3(SO3d, t)
print("SE3: ")
SE3.to_homogenous_matrix()

SE3: 


⎡0  -1  0  1⎤
⎢           ⎥
⎢1  0   0  0⎥
⎢           ⎥
⎢0  0   1  0⎥
⎢           ⎥
⎣0  0   0  1⎦

In [6]:
# Look at the lie algebra (V6)
se3 = sf.V6(SE3.to_tangent())
se3.transpose()  # I noticed the order is different for rho and phi

⎡      π  π            ⎤
⎢0  0  ─  ─  -0.25⋅π  0⎥
⎣      2  4            ⎦

In [7]:
def make_skew(xi: sf.Vector6)->sf.M44:
    phi = sf.Vector3(xi[:3])
    rho = sf.Vector3(xi[3:])
    phi_hat = phi.skew_symmetric(phi)
    return sf.M44.block_matrix([
        [phi_hat, rho],
        [sf.Vector3.zeros(3,1).transpose(), sf.Vector1(0)] 
    ])

def skew_to_vec6(M: sf.M44)->sf.Vector6:
    rot_part = M[:3,:3]
    vec = skew_to_vec(rot_part)
    rho = M[:3, 3]
    return sf.Vector6.block_matrix([[rho], [vec]])

se3_hat = make_skew(se3)
se3_hat

⎡   -π         π   ⎤
⎢0  ───  0     ─   ⎥
⎢    2         4   ⎥
⎢                  ⎥
⎢π                 ⎥
⎢─   0   0  -0.25⋅π⎥
⎢2                 ⎥
⎢                  ⎥
⎢0   0   0     0   ⎥
⎢                  ⎥
⎣0   0   0     0   ⎦

In [8]:
skew_to_vec6(se3_hat).transpose()

⎡π                    π⎤
⎢─  -0.25⋅π  0  0  0  ─⎥
⎣4                    2⎦

In [25]:
# Finally the update:
update_se3 = sf.V6()
# NOTE that the order of the vector is different than the book
# the translation is the second part here. 
update_se3[3] = 1e-4
SE3_updated = sf.Pose3_SE3.from_tangent(update_se3, .00000001)*SE3
display(SE3_updated.to_homogenous_matrix())


⎡ 0   -1.0  0  1.0001⎤
⎢                    ⎥
⎢1.0   0    0    0   ⎥
⎢                    ⎥
⎢ 0    0    1    0   ⎥
⎢                    ⎥
⎣ 0    0    0    1   ⎦